In [5]:
! git clone https://github.com/Icon1cc/NLP-Polimi-Project.git

fatal: destination path 'NLP-Polimi-Project' already exists and is not an empty directory.


In [3]:
pip install textstat


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q transformers datasets


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.util import ngrams
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from textstat import flesch_reading_ease
from datasets import load_dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertForQuestionAnswering
from tokenizers import BertWordPieceTokenizer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
from transformers import TrainingArguments, Trainer

/usr/local/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [9]:
# Load the dataset
dataset = load_dataset('arrow', data_files='/kaggle/working/NLP-Polimi-Project/Practice Models/Rishabh/dataset/train/data-00000-of-00001.arrow')
df = dataset['train'].to_pandas()
df = df.iloc[:, :-1]

In [5]:
#dataset length
len(df)

33955

In [11]:
df.head()

,input,output
0,What is the relationship between very low Mg2+...,Very low Mg2+ levels correspond to low PTH lev...
1,What leads to genitourinary syndrome of menopa...,Low estradiol production leads to genitourinar...
2,What does low REM sleep latency and experienci...,Low REM sleep latency and experiencing halluci...
3,What are some possible causes of low PTH and h...,"PTH-independent hypercalcemia, which can be ca..."
4,How does the level of anti-müllerian hormone r...,The level of anti-müllerian hormone is directl...


In [32]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
model_name = 'atharvamundada99/bert-large-question-answering-finetuned-legal'

In [13]:
BERT = BertForQuestionAnswering.from_pretrained(model_name)

In [14]:
#print BERT model
print(BERT)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [15]:
#Check which of the parameters are trainable
for param in BERT.parameters():
    print(param.size(), param.requires_grad)

torch.Size([30522, 1024]) True
torch.Size([512, 1024]) True
torch.Size([2, 1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([4096, 1024]) True
torch.Size([4096]) True
torch.Size([1024, 4096]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([4096, 1024]) True
torch.Size([4096]) True
torch.Size([1024, 4096]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024]) True
torch.Size([1024, 1024]) 

In [19]:
#load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [33]:
#create a list of dictionaries with all the samples in the train and test data sets
train_data_dict = [{'question': row['input'], 'answer': row['output']} for _, row in train_data.iterrows()]
test_data_dict = [{'question': row['input'], 'answer': row['output']} for _, row in test_data.iterrows()]

In [34]:
#convert in hugging face API format
train_data = Dataset.from_list(train_data_dict)
test_data = Dataset.from_list(test_data_dict)

In [35]:
data = DatasetDict()
data['train'] = train_data
data['validation'] = test_data
data['test'] = test_data

In [48]:
def tokenize_function(examples):
    return tokenizer(examples["question"], padding=True, truncation=True)

tokenized_data = data.map(tokenize_function, batched=True)

Map: 100%|██████████| 6791/6791 [00:00<00:00, 18548.88 examples/s]


In [54]:
print(train_data[4])

{'question': 'To which drug class do ipratropium and tiotropium belong?', 'answer': 'Ipratropium and tiotropium belong to the drug class of inhaled M3 antimuscarinics.'}


In [52]:
#vocabulary size of BERT
print("vocabulary size: ", len(tokenizer.vocab))

vocabulary size:  30522


# TRAINING 

In [56]:
for param in BERT.parameters():
    param.requires_grad = False

In [1]:
training_args = TrainingArguments(
    "cool_trainer_name", 
    per_device_train_batch_size=16
)

NameError: name 'TrainingArguments' is not defined